<a href="https://colab.research.google.com/github/thamizhannal/eva5/blob/master/Quiz_S9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Session9 Quiz


In [1]:
from datetime import datetime
import datetime
print("Current Date/Time: ",  datetime.datetime.now())

Current Date/Time:  2020-09-26 18:17:41.466449


In [2]:
pwd

'/content'

In [3]:

!git clone https://github.com/theschoolof-ai/JEDI.git

fatal: destination path 'JEDI' already exists and is not an empty directory.


In [4]:
cd /content/JEDI/tsai.jedi

/content/JEDI/tsai.jedi


In [22]:
ls Models/

__pycache__/  QuizDNN.py  S6.py  S7.py  S8_resnet.py  S9_resnet.py


In [75]:
# QuizDNN.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append(".")
from batchnorm import GhostBatchNorm
import config


def model_summary(model_, input_):
    from torchsummary import summary
    use_cuda = config.use_cuda
    device = config.device
    arch = model_.to(device)
    return summary(arch, input_)

'''
x1 = Input
x2 = Conv(x1)
x3 = Conv(x1 + x2)
x4 = MaxPooling(x1 + x2 + x3)
x5 = Conv(x4)
x6 = Conv(x4 + x5)
x7 = Conv(x4 + x5 + x6)
x8 = MaxPooling(x5 + x6 + x7)

x9 = Conv(x8)
x10 = Conv (x8 + x9)
x11 = Conv (x8 + x9 + x10)
x12 = GAP(x11)
x13 = FC(x12)

'''

class Net(nn.Module):

    # Function to fetch GhostBatchNorm
    # flag = 1 for GhostBatchNorm and flag = 0 for batchnorm2d
    def batch_norm(self, channels):
        return nn.BatchNorm2d(channels)

    def __init__(self):
        
        super(Net, self).__init__()

        # block1
        
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=64, kernel_size=3, padding=1)  # o/p size:32; rf: 3
        self.Batchnorm1 = self.batch_norm(64)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)  # o/p size: 32; rf: 5
        self.Batchnorm2 = self.batch_norm(64)

        # MP1 block
        self.pool1 = nn.MaxPool2d(2, 2)  # o/p size: 16; rf: 10
        
		
   	    # block2
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size= 3,padding=1)  # o/p size: 16; rf: 12
        self.Batchnorm3 = self.batch_norm(128)
        self.dp3 = nn.Dropout(p=0.10)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,padding=1)  # o/p size: 16; rf: 14
        self.Batchnorm4 = self.batch_norm(128)
        self.dp4 = nn.Dropout(p=0.10)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,padding=1)  # o/p size: 16; rf: 18
        self.Batchnorm5 = self.batch_norm(128)
        self.dp5 = nn.Dropout(p=0.10)

		    # transition block
        self.pool2 = nn.MaxPool2d(2, 2)  # o/p size: 8; rf: 36
        
        self.conv6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3,padding=1)  # o/p size: 8; rf: 38
        self.Batchnorm3 = self.batch_norm(256)
        self.dp3 = nn.Dropout(p=0.10)
        self.conv7 = nn.Conv2d(in_channels=256,out_channels= 256, kernel_size=3,padding=1)  # o/p size: 8; rf: 40
        self.Batchnorm4 = self.batch_norm(256)
        self.dp4 = nn.Dropout(p=0.10)
        self.conv8 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3,padding=1)  # o/p size: 8; rf: 42
        self.Batchnorm5 = self.batch_norm(256)
        self.dp5 = nn.Dropout(p=0.10)
		
		
        # gap and 1X1
        self.conv9_avgp = nn.AvgPool2d(kernel_size=8)  # o/p size: 1; rf: 30
        self.fc1   = nn.Linear(256*8*8, 10)
       

    def forward(self, x):
        

        x2 = self.conv1(x)
        x2 = F.relu(x2)
        x2 = self.Batchnorm1(x2)

        x3 = self.conv2(x2)
        x3 = F.relu(x3)
        x3 = self.Batchnorm2(x3)

        x4 = self.pool1(x3)

        x5 = self.conv1(x4)
        x5 = F.relu(x5)
        x5 = self.Batchnorm1(x5)

        x6 = self.conv2(x5)
        x6 = F.relu(x6)
        x6 = self.Batchnorm2(x6)
        
        x7 = self.conv2(x6)
        x7 = F.relu(x7)
        x7 = self.Batchnorm2(x7)
        
        x8 = self.pool2(x7)
		
        x9 = self.conv1(x8)
        x9 = F.relu(x9)
        x9 = self.Batchnorm1(x9)

        x10 = self.conv2(x9)
        x10 = F.relu(x10)
        x10 = self.Batchnorm2(x10)
        
        x11 = self.conv2(x10)
        x11 = F.relu(x11)
        x11 = self.Batchnorm2(x11)

        x12 = self.conv9_avgp(x11)

        x13 = self.fc1(x12)

        x = x13.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [76]:
print(config.input_size_CIFAR10)
model_summary(Net(), config.input_size_CIFAR10)

(3, 32, 32)


RuntimeError: ignored

In [7]:
import sys
sys.path.append('Models/')
from S8_resnet import ResNet18
from S7 import model_summary

In [8]:
! pip install albumentations==0.4.6

In [9]:
import torch
import config
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from Engine_train_test import train, test
from dataloader import train_loader_CIFAR10,test_loader_CIFAR10

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified


In [10]:
config.input_size_CIFAR10

(3, 32, 32)

In [33]:
#from Models import QuizDNN
#from S7 import model_summary
config.input_size_CIFAR10

(3, 32, 32)

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BatchNorm(nn.BatchNorm2d):
    def __init__(self, num_features, eps=1e-05, momentum=0.1, weight=True, bias=True):
        super().__init__(num_features, eps=eps, momentum=momentum)
        self.weight.data.fill_(1.0)
        self.bias.data.fill_(0.0)
        self.weight.requires_grad = weight
        self.bias.requires_grad = bias
class GhostBatchNorm(BatchNorm):
    def __init__(self, num_features, num_splits, **kw):
        super().__init__(num_features, **kw)
        self.num_splits = num_splits
        self.register_buffer('running_mean', torch.zeros(num_features * self.num_splits))
        self.register_buffer('running_var', torch.ones(num_features * self.num_splits))

    def train(self, mode=True):
        if (self.training is True) and (mode is False):  # lazily collate stats when we are going to use them
            self.running_mean = torch.mean(self.running_mean.view(self.num_splits, self.num_features), dim=0).repeat(
                self.num_splits)
            self.running_var = torch.mean(self.running_var.view(self.num_splits, self.num_features), dim=0).repeat(
                self.num_splits)
        return super().train(mode)

    def forward(self, input):
        N, C, H, W = input.shape
        if self.training or not self.track_running_stats:
            return F.batch_norm(
                input.view(-1, C * self.num_splits, H, W), self.running_mean, self.running_var,
                self.weight.repeat(self.num_splits), self.bias.repeat(self.num_splits),
                True, self.momentum, self.eps).view(N, C, H, W)
        else:
            return F.batch_norm(
                input, self.running_mean[:self.num_features], self.running_var[:self.num_features],
                self.weight, self.bias, False, self.momentum, self.eps)

#depth wise implementation
class depthwise_separable_conv(nn.Module):
    def __init__(self, nin, nout,kernel_size_ = 3,  padding_ = 1,kernels_per_layer=1):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin*kernels_per_layer, kernel_size=kernel_size_, padding=padding_, groups=nin)
        self.pointwise = nn.Conv2d(nin * kernels_per_layer, nout, kernel_size=1)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out


In [51]:
model_summary(Net(1), config.input_size_CIFAR10)

RuntimeError: ignored

In [17]:

from __future__ import print_function
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import sys

input_size_CIFAR10 = (3, 32, 32)

print(model_summary(Net(flag=1), config.input_size_CIFAR10))

RuntimeError: ignored

In [19]:
from datetime import datetime
import datetime
print("Current Date/Time: ",  datetime.datetime.now())

Current Date/Time:  2020-09-26 18:25:00.733012
